# Human Recognition System

In [ ]:
import cv2 as cv
import numpy as np
import os
import face_recognition
import speech_recognition as sr
import webbrowser as wb

# voice Detection

In [ ]:
def main():
    r = sr.Recognizer()

    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source)

        print("Please Say Your Name: ")

        audio = r.listen(source)

        print("Recognizing Now .... ")

        # recognize speech using google

        try:
            print("You have said \n" + r.recognize_google(audio))
            print("Audio Recorded Successfully \n ")


        except Exception as e:
            print("Error :  Voice is not Cleared plzz speak again....\n")
            
        # write audio
        with open("voice_database/recorded.wav", "wb") as f:
            f.write(audio.get_wav_data())
        return r.recognize_google(audio)

name=main()

## Add a New User

In [ ]:
img_dir = r'C:\Users\HP\jupyterProjects\face-recognition-system\face_database'
os.chdir(img_dir)

# used to capture an image.
cam = cv.VideoCapture(0)

cv.namedWindow("web cam")

img_counter = 0

while True:
    ret, frame = cam.read()
    if not ret:
        print("failed to grab frame")
        break
    cv.imshow("web cam", frame)

    k = cv.waitKey(1)
    if k%256 == 27:
        # ESC pressed`
        print("Escape hit, closing...")
        break
    elif k%256 == 32:
        # SPACE pressed
        img_name = f"{name}_{img_counter}.png"
        cv.imwrite(img_name, frame)
        print("{} written!".format(img_name))
        img_counter += 1

cam.release()
cv.destroyAllWindows()

# FACIAL ENCODING

In [ ]:
os.chdir(r'C:\Users\HP\jupyterProjects\face-recognition-system')
path = r'face_database'
images = []
classNames = []
mylist = os.listdir(path)


for cl in mylist:
    curImg = cv.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])


def findencodings(images):
    encodelist = []
    for img in images:
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodelist.append(encode)
    return encodelist


encodeListKnown = findencodings(images)
print('Encoding Complete..')
print('Total Encoded Elements: ', len(encodeListKnown))

# FACE CLASSIFICATION

In [ ]:
cap = cv.VideoCapture(0)

while True:
    success, img = cap.read()
    imgS = cv.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv.cvtColor(imgS, cv.COLOR_BGR2RGB)
    facesCurFrame = face_recognition.face_locations(imgS)
    encodeCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)

    for encodeFace, faceLoc in zip(encodeCurFrame, facesCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
        matchIndex = np.argmin(faceDis)

        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv.FILLED)
            cv.putText(img, name, (x1 + 6, y2 - 6), cv.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
    cv.imshow('webcam', img)
    k = cv.waitKey(1)
    if k%256 == 27:
        # ESC pressed`
        print("Escape hit, closing...")
        break

cap.release()
cv.destroyAllWindows()